In [3]:
!export HF_ENDPOINT=https://hf-mirror.com

In [1]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

model_path = "/mnt/workspace/llava-v1.5-7b"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path)
)

/opt/conda/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-12-03 18:13:47,809] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards: 100%|██████████| 2/2 [01:15<00:00, 37.71s/it]


In [10]:
from llava.conversation import conv_templates
from llava.mm_utils import tokenizer_image_token
from PIL import Image
import torch

In [5]:
image_file = '/mnt/workspace/LLaVA-1.5/notebook/photo.jpg'

In [14]:
from dataclasses import dataclass

@dataclass
class Args:
    temperature: float = 0.2
    top_p: float = None
    num_beams: int = 1

args = Args()
print(args)

Args(temperature=0.2, top_p=None, num_beams=1)


In [15]:
DEFAULT_IMAGE_TOKEN = "<image>"
IMAGE_TOKEN_INDEX = -200
qs = 'describe this image'
qs = DEFAULT_IMAGE_TOKEN + '\n' + qs
conv = conv_templates['llava_v1'].copy()
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()

image = Image.open(image_file)
image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0]

stop_str = '<s>'
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=image_tensor.unsqueeze(0).half().cuda(),
        do_sample=True if args.temperature > 0 else False,
        temperature=args.temperature,
        top_p=args.top_p,
        num_beams=args.num_beams,
        # no_repeat_ngram_size=3,
        max_new_tokens=1024,
        use_cache=True)

input_token_len = input_ids.shape[1]
n_diff_input_output = (input_ids != output_ids[:, :input_token_len]).sum().item()
if n_diff_input_output > 0:
    print(f'[Warning] {n_diff_input_output} output_ids are not the same as the input_ids')
outputs = tokenizer.batch_decode(output_ids[:, input_token_len:], skip_special_tokens=True)[0]
outputs = outputs.strip()
if outputs.endswith(stop_str):
    outputs = outputs[:-len(stop_str)]
outputs = outputs.strip()
print(outputs)

The image depicts a small, clean, and well-organized bedroom with a bed and a desk. The bed is positioned against the left wall, and the desk is located on the right side of the room. A chair is placed in front of the desk, providing a comfortable seating area for work or study. 

In addition to the bed and desk, there is a TV mounted on the wall above the bed, and a laptop is placed on the desk. A few books can be seen on the desk, and a cup is located near the right edge of the desk. A teddy bear is also present in the room, adding a touch of warmth and personality to the space.
